# Split Apply Combine

Often times, we will want to split a dataset based on a key value in a column, and do something within this group, and return this information for each group.  We will often see this referred to as the **split-apply-combine** paradigm.  With Pandas, we will often use the `groupby` method to carry out the splitting piece of this. 

**OBJECTIVES**:

- Use `groupby` to split dataset into groups, and groups within groups
- Apply different methods to `groupby` objects
- Use aggregation methods on `groupby` objects
- Write and use functions of our own on `groupby` objects with `apply`

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

In [2]:
df = pd.DataFrame({'street': ['a', 'a', 'b', 'b', 'a'],
                  'agent': ['one', 'one', 'two', 'one', 'two'],
                  'sq_ft': np.random.randint(1000, 4500, 5),
                  'price': np.random.randint(100000, 600000, 5)})

In [3]:
df.head()

,street,agent,sq_ft,price
0,a,one,1736,217636
1,a,one,1019,458842
2,b,two,1227,241189
3,b,one,4047,137327
4,a,two,3069,340087


### `groupby` Basics

To begin, it is important to recognize that the result of using the `groupby` method is a `groupby` object.  This is an object that has simply grouped the data according to our input.  From here, we can select columns like usual and apply basic quantitative methods.  If we wanted to know the average square footage by agent in our mini-data above, we select the column we are concerned with (`df['sq_ft']`), and group this by agent(`.groupby(df['agent'])`).  We are returned an object for which we can then apply the mean to.  Below, we save our grouped data as `grouped`, and can subsequently perform a variety of methods.

In [4]:
#groupby returns a groupby object
df['sq_ft'].groupby(df['agent'])

In [5]:
grouped = df['sq_ft'].groupby(df['agent'])

In [6]:
#we can aggregate the resulting groups
grouped.mean()

agent
one    2267.333333
two    2148.000000
Name: sq_ft, dtype: float64

In [7]:
#using any aggregate function
grouped.count()

agent
one    3
two    2
Name: sq_ft, dtype: int64

In [8]:
grouped.describe()

,count,mean,std,min,25%,50%,75%,max
agent,,,,,,,,
one,3.0,2267.333333,1582.381854,1019.0,1377.5,1736.0,2891.5,4047.0
two,2.0,2148.000000,1302.490691,1227.0,1687.5,2148.0,2608.5,3069.0


We can pass more than one value to group by, and will be returned an object that has two levels of indices.  For example, if we wanted to know:

$$
\textit{What is the average price for each agent by street?}
$$

We will select the price column, group the data by agent and street, and apply the mean method to this.  Notice that there are two levels of indices, *agent* and *street*.   

In [9]:
means = df['price'].groupby([df['agent'], df['street']]).mean()

In [10]:
means

agent  street
one    a         338239
       b         137327
two    a         340087
       b         241189
Name: price, dtype: int64

In [11]:
means.keys()

MultiIndex([('one', 'a'),
            ('one', 'b'),
            ('two', 'a'),
            ('two', 'b')],
           names=['agent', 'street'])

We can return a dataframe that unstacks the levels with the `unstack` method. 

In [12]:
means.unstack()

street,a,b
agent,,
one,338239,137327
two,340087,241189


In [13]:
means[0]

338239

In [14]:
means[3]

241189

In [15]:
means.shape

(4,)

### Example I 

Using our tips dataset example, we can explore a few straightforward questions using `groupby()`.  

1. What is the average tip by gender?
2. What is the max tip on each day?
3. What is the min tip on each day by time?
4. What is the average bill by party size?

In [16]:
tips = sns.load_dataset('tips')

In [17]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [18]:
#average tip by gender


In [19]:
#max tip each day


In [20]:
#min tip on each day by time


In [21]:
#What is the average bill by party size?


### Groupby Objects

The results of the groupby object are iterable.  For example, if we group the tips data by smoker, we get back the group and the elements of this group.  For multiple indicies of groups, we pass these as tuples.

In [22]:
for name, group in tips.groupby(['smoker']):
    print(name)
    print(group)

Yes
     total_bill   tip     sex smoker  day    time  size
56        38.01  3.00    Male    Yes  Sat  Dinner     4
58        11.24  1.76    Male    Yes  Sat  Dinner     2
60        20.29  3.21    Male    Yes  Sat  Dinner     2
61        13.81  2.00    Male    Yes  Sat  Dinner     2
62        11.02  1.98    Male    Yes  Sat  Dinner     2
..          ...   ...     ...    ...  ...     ...   ...
234       15.53  3.00    Male    Yes  Sat  Dinner     2
236       12.60  1.00    Male    Yes  Sat  Dinner     2
237       32.83  1.17    Male    Yes  Sat  Dinner     2
240       27.18  2.00  Female    Yes  Sat  Dinner     2
241       22.67  2.00    Male    Yes  Sat  Dinner     2

[93 rows x 7 columns]
No
     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.

In [23]:
for (n1, n2), group in tips.groupby(['sex', 'smoker']):
    print(n1, n2)
    print(group)

Male Yes
     total_bill    tip   sex smoker   day    time  size
56        38.01   3.00  Male    Yes   Sat  Dinner     4
58        11.24   1.76  Male    Yes   Sat  Dinner     2
60        20.29   3.21  Male    Yes   Sat  Dinner     2
61        13.81   2.00  Male    Yes   Sat  Dinner     2
62        11.02   1.98  Male    Yes   Sat  Dinner     2
63        18.29   3.76  Male    Yes   Sat  Dinner     4
69        15.01   2.09  Male    Yes   Sat  Dinner     2
76        17.92   3.08  Male    Yes   Sat  Dinner     2
80        19.44   3.00  Male    Yes  Thur   Lunch     2
83        32.68   5.00  Male    Yes  Thur   Lunch     2
90        28.97   3.00  Male    Yes   Fri  Dinner     2
95        40.17   4.73  Male    Yes   Fri  Dinner     4
96        27.28   4.00  Male    Yes   Fri  Dinner     2
97        12.03   1.50  Male    Yes   Fri  Dinner     2
98        21.01   3.00  Male    Yes   Fri  Dinner     2
105       15.36   1.64  Male    Yes   Sat  Dinner     2
106       20.49   4.06  Male    Yes   S

In [24]:
#one-liner to create dictionary of groups
pieces = dict(list(tips.groupby('sex')))

In [25]:
pieces['Male'].head()

,total_bill,tip,sex,smoker,day,time,size
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2


In [26]:
pieces['Female'].head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
11,35.26,5.00,Female,No,Sun,Dinner,4
14,14.83,3.02,Female,No,Sun,Dinner,2
16,10.33,1.67,Female,No,Sun,Dinner,3


##### PROBLEM

1. Create a group of `DataFrame` objects from our tips data based on days.  Display the head of Sunday's data.

### Data Aggregation

Here, we are interested in using the aggreate function to apply functions that we have developed which return scalar values.  For example, we can write a function that determines the range of tips on a given day.  We are taking all of the values for tips by group, and finding a single value for this.  Similarly, we could do something like investigate the quantiles of each group.

In [33]:
grouped = tips.groupby('day')

In [34]:
def ranger(df):
    return df.max() - df.min()

In [35]:
grouped.describe()

total_bill                                                             \
          count       mean       std   min      25%    50%      75%    max   
day                                                                          
Thur       62.0  17.682742  7.886170  7.51  12.4425  16.20  20.1550  43.11   
Fri        19.0  17.151579  8.302660  5.75  12.0950  15.38  21.7500  40.17   
Sat        87.0  20.441379  9.480419  3.07  13.9050  18.24  24.7400  50.81   
Sun        76.0  21.410000  8.832122  7.25  14.9875  19.63  25.5975  48.17   

       tip            ...                 size                                \
     count      mean  ...     75%    max count      mean       std  min  25%   
day                   ...                                                      
Thur  62.0  2.771452  ...  3.3625   6.70  62.0  2.451613  1.066285  1.0  2.0   
Fri   19.0  2.734737  ...  3.3650   4.73  19.0  2.105263  0.567131  1.0  2.0   
Sat   87.0  2.993103  ...  3.3700  10.00  87.0  2.517241  0.819275  1.0  2.0   
Sun   76.0  3.255132  ...  4.0000   6.50  76.0  2.842105  1.007341  2.0  2.0   

                     
      50%  75%  max  
day                  
Thur  2.0  2.0  6.0  
Fri   2.0  2.0  4.0  
Sat   2.0  3.0  5.0  
Sun   2.0  4.0  6.0  

[4 rows x 24 columns]

In [36]:
grouped.agg(ranger)

,total_bill,tip,size
day,,,
Thur,35.60,5.45,5
Fri,34.42,3.73,3
Sat,47.74,9.00,4
Sun,40.92,5.49,4


In [42]:
grouped['total_bill'].quantile(0.8)

day
Thur    22.440
Fri     22.594
Sat     26.554
Sun     29.850
Name: total_bill, dtype: float64

In [44]:
grouped['tip'].quantile(0.9)

day
Thur    4.920
Fri     4.060
Sat     4.802
Sun     5.035
Name: tip, dtype: float64

We can add a tip percentage column as follows.

In [45]:
tips['pct_tip'] = tips['tip']/tips['total_bill']

In [46]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,pct_tip
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [47]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg('mean')

sex     smoker
Male    Yes       0.152771
        No        0.160669
Female  Yes       0.182150
        No        0.156921
Name: pct_tip, dtype: float64

In [48]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(['mean', 'std', ranger])

mean       std    ranger
sex    smoker                              
Male   Yes     0.152771  0.090588  0.674707
       No      0.160669  0.041849  0.220186
Female Yes     0.182150  0.071595  0.360233
       No      0.156921  0.036421  0.195876

In [49]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(
    [('Average Tipping Percentage','mean'), ('Standard Deviation', 'std'), ('Range', ranger)])

Average Tipping Percentage  Standard Deviation     Range
sex    smoker                                                          
Male   Yes                       0.152771            0.090588  0.674707
       No                        0.160669            0.041849  0.220186
Female Yes                       0.182150            0.071595  0.360233
       No                        0.156921            0.036421  0.195876

In [50]:
funcs = ['mean', 'max', 'min', 'std']

In [51]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(funcs)

mean       max       min       std
sex    smoker                                        
Male   Yes     0.152771  0.710345  0.035638  0.090588
       No      0.160669  0.291990  0.071804  0.041849
Female Yes     0.182150  0.416667  0.056433  0.071595
       No      0.156921  0.252672  0.056797  0.036421

In [ ]:
#overwrite results with new
#column names
func_named = [('Average', 'mean'), ('Maximum', 'max')]

In [ ]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(func_named)

In [ ]:
#pass different aggregation functions
tips.groupby(['sex', 'smoker']).agg({'tip': [np.min, np.max], 'pct_tip': np.max})

In [ ]:
#pass multiple aggregation functions
tips.groupby(['sex', 'smoker']).agg({'tip': [np.min, np.mean], 'pct_tip': np.max})

In [ ]:
#ignore index labels
tips.groupby(['sex', 'smoker'], as_index=False).agg({'tip': [np.min, np.mean], 'pct_tip': np.max})

##### Problem

Read in the `gapminder_all` data.  Use the `groupby` and `agg` methods to create a new summary dataframe that contains columns for *average*, *minimum*, and *maximum* gdp and life expectancy for the year 2007.

### `apply`

Beyond aggregation functions, we can apply a more general call to functions that don't necessarily return a scalar value.  For example, suppose we wanted to pass a function that will take the top 5 tip percentages.  Then, we can apply this to different groupings of the data.  Because our function takes a column argument as well as a top number, we can call these using the `apply` method also.

In [52]:
def top(df, n=5, column='pct_tip'):
    return df.sort_values(by = column)[-n:]

In [53]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   pct_tip
smoker                                                                   
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990

In [54]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   pct_tip
smoker                                                                   
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990

In [55]:
tips.groupby(['smoker', 'day']).apply(top, n=2, column = 'total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
Yes    Thur 83        32.68   5.00    Male    Yes  Thur   Lunch     2   
            197       43.11   5.00  Female    Yes  Thur   Lunch     4   
       Fri  90        28.97   3.00    Male    Yes   Fri  Dinner     2   
            95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  102       44.30   2.50  Female    Yes   Sat  Dinner     3   
            170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  184       40.55   3.00    Male    Yes   Sun  Dinner     2   
            182       45.35   3.50    Male    Yes   Sun  Dinner     3   
No     Thur 85        34.83   5.17  Female     No  Thur   Lunch     4   
            142       41.19   5.00    Male     No  Thur   Lunch     5   
       Fri  91        22.49   3.50    Male     No   Fri  Dinner     2   
            94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  59        48.27   6.73    Male     No   Sat  Dinner     4   
            212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  112       38.07   4.00    Male     No   Sun  Dinner     3   
            156       48.17   5.00    Male     No   Sun  Dinner     6   

                  pct_tip  
smoker day                 
Yes    Thur 83   0.152999  
            197  0.115982  
       Fri  90   0.103555  
            95   0.117750  
       Sat  102  0.056433  
            170  0.196812  
       Sun  184  0.073983  
            182  0.077178  
No     Thur 85   0.148435  
            142  0.121389  
       Fri  91   0.155625  
            94   0.142857  
       Sat  59   0.139424  
            212  0.186220  
       Sun  112  0.105070  
            156  0.103799

In [56]:
tips.groupby('smoker', group_keys = False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,pct_tip
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990


In [57]:
tips.groupby('smoker')['pct_tip'].describe(percentiles = [])

,count,mean,std,min,50%,max
smoker,,,,,,
Yes,93.0,0.163196,0.085119,0.035638,0.153846,0.710345
No,151.0,0.159328,0.039910,0.056797,0.155625,0.291990


In [58]:
tips.groupby('smoker')['pct_tip'].describe(percentiles = []).unstack()

       smoker
count  Yes        93.000000
       No        151.000000
mean   Yes         0.163196
       No          0.159328
std    Yes         0.085119
       No          0.039910
min    Yes         0.035638
       No          0.056797
50%    Yes         0.153846
       No          0.155625
max    Yes         0.710345
       No          0.291990
dtype: float64

##### PROBLEM

Using the `gapminder_all.csv` data, read in the data and create a function to generate a new column `color` based on the name of the continent.  One approach would be -- create a dictionary of values, write a function to lookup the key as continent and color as value, apply this.  

### Further Reading

- [Pandas `groupby` documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html)
